In [2]:
import os
import sys
import pandas as pd
import glob
import re
import copy
import numpy as np
import nltk
import random

from spacy.lang.nl import Dutch

from collections import Counter

In [3]:
os.getcwd()

'/srv/data/Stage_LR'

# Working with dump from file

In [4]:
filepath = os.getcwd() + '/brieven'
print(filepath)

/srv/data/Stage_LR/brieven


In [5]:
info_dict = dict()
info_list = []
path = '/srv/data/Stage_LR/brieven'
counter = 0

for filename in glob.glob(os.path.join(path, '*.txt')):
   #print(filename)
    with open(filename, 'r') as f:
        f = f.read()
        if ';' in f:
            f.replace(';', ':')
        info_dict['filename'] = filename
        info_dict['text'] = f
        info_dict['len_text'] = len(f)
        info_dict['Speech Act'] = ""
        info_list.append(info_dict)
        info_dict = dict()
        
   #break


In [6]:
df = pd.DataFrame(info_list)
df = df.drop_duplicates()
df = df[df['len_text'] >= 70]
df = df.sort_values(ascending=True, by=['len_text'])
#df.to_csv("txt_df.csv", sep='\t')

# Working with excel dataframe

In [7]:
df_new = pd.read_excel('results.xlsx', engine='openpyxl')


In [8]:
print(df_new[df_new["FileCode"] == 'DIV'].shape)

(4, 5)


In [9]:
df_new = df_new.sort_values(by='Text', key=lambda x: x.str.len(), ascending=False)
df_new["len_text"] = df_new["Text"].str.len()
df_new["whitespace_count"] = df_new["Text"].str.count(' ')
df_new["newline_count"] = df_new["Text"].str.count('\n')
df_new["perc_newline"] = round((df_new["newline_count"] / df_new["len_text"]) *100, 2)
df_new["perc_whitespace"] = round((df_new["whitespace_count"] / df_new["len_text"]) *100, 2) ##determine percentage of whitespace to use for filtering later
df_new["total_nr_sent"] = 1 ## will be used later
df_new["sent_index_in_tb"] = 0 ## will be used later
print("Initial shape", df_new.shape)


Initial shape (3690, 12)


In [10]:
df_new = df_new.drop_duplicates(subset='Text', inplace=False) ##drop duplicate text blocks
print("Without duplicates: ", df_new.shape)

#df_new.to_csv("inspect.csv", ';')
df_new = df_new[df_new["perc_whitespace"] < 25] ##only keep text segments with max 25% whitespace
df_new = df_new[df_new["len_text"] > 30] ##only keep texts with more than 10 characters
print("Less than 25% whitespace and a total length of more than 30 caracters: ", df_new.shape)

df_new = df_new.sort_values(ascending=False, by=['perc_whitespace'])

#df_new.to_csv("Full_Text_df.csv") ##write to csv

print(df_new.columns)
df_new.shape


Without duplicates:  (3318, 12)
Less than 25% whitespace and a total length of more than 30 caracters:  (2891, 12)
Index(['FileCode', 'FilePath', 'FileName', 'FileCaption', 'Text', 'len_text',
       'whitespace_count', 'newline_count', 'perc_newline', 'perc_whitespace',
       'total_nr_sent', 'sent_index_in_tb'],
      dtype='object')


(2891, 12)

## Examples

In [11]:
counter = 0
for text in df_new["Text"]:
    counter += 1
    #print(text)
    #print('-----------------------------------------------------------------------------------------------')
    #print()
    if counter == 4:
        print(text[:100], '(...)')
        break




Betreft: Ondernemerschap en belastingplicht voor de omzetbelasting (btw)


    
    
    
    
    
 (...)


## Clean text: delete most of the \s

In [12]:
result = []
cleaned_text = []

for content in df_new["Text"]:
    result = re.findall(re.compile(r'\n\s*\n\s*\n\s*\n\s\n*\s*[\n*\s*]*'), content) 
    #result.extend(re.findall(re.compile(r'...+'), content))
    new_text = content
    for i in range(0,len(result)):
        new_text = new_text.replace(result[i], '')
    new_text = new_text.replace(';', ':') ##to use ; as separator later
    cleaned_text.append(new_text.replace('\n', ' '))

df_new["cleaned_text"] = cleaned_text




In [13]:
counter = 0

for text in df_new["cleaned_text"]:
    #print(text)
    counter += 1
    #print('****************************************************************************************************************************')
    if counter == 4:
        print(text[:100], '(...)')
        break

Betreft: Ondernemerschap en belastingplicht voor de omzetbelasting (btw)U hebt u aangemeld als onder (...)


In [14]:
print(df_new[df_new["FileCode"] == 'OB'].shape)

(325, 13)


## Split up textblocks in individual sentences

In [15]:
print(df_new.columns)
print(df_new.shape)

list_df = df_new.values.tolist()


Index(['FileCode', 'FilePath', 'FileName', 'FileCaption', 'Text', 'len_text',
       'whitespace_count', 'newline_count', 'perc_newline', 'perc_whitespace',
       'total_nr_sent', 'sent_index_in_tb', 'cleaned_text'],
      dtype='object')
(2891, 13)


In [16]:
index = -1

final_df_list = []

nlp = Dutch()
nlp.add_pipe(nlp.create_pipe('sentencizer'))




for rows in list_df:
    if '.' in rows[-1]: #if . in text, strip..
        test = [str(sent).strip() for sent in nlp(rows[-1]).sents]
        length = len(test)
        length_row = copy.deepcopy(rows)
        length_row[-3] = length
        for i in range(0, length-1):
            new_row = copy.deepcopy(length_row) ##to prevent the variables to get overwritten in loop
            new_row[-1] = test[i]
            if i == 2:
                new_row[-2] = 1
            else:
                new_row[-2] = i


            xx = new_row

            final_df_list.append(xx)


    else:
        final_df_list.append(rows)
        

    
    
new_new_df = pd.DataFrame(final_df_list, columns=df_new.columns)

new_new_df.shape


(37403, 13)

In [17]:
new_new_df[new_new_df["FileCode"] == 'DIV'].shape

(36, 13)

In [18]:
no_dup_df = new_new_df.drop_duplicates(subset='cleaned_text', inplace=False)

no_dup_df.shape

no_dup_df

,FileCode,FilePath,FileName,FileCaption,Text,len_text,whitespace_count,newline_count,perc_newline,perc_whitespace,total_nr_sent,sent_index_in_tb,cleaned_text
0,IV,IV/NII/tekstblokken,NII055_00_NL.xml,NII055_00_NL,\n \n \n \n \n ...,1539.0,384.0,35.0,2.27,24.95,15,0,U hebt bij de Nederlandse Belastingdienst nog ...
1,IV,IV/NII/tekstblokken,NII055_00_NL.xml,NII055_00_NL,\n \n \n \n \n ...,1539.0,384.0,35.0,2.27,24.95,15,1,Op de bijlage bij deze brief vindt u een overz...
2,IV,IV/NII/tekstblokken,NII055_00_NL.xml,NII055_00_NL,\n \n \n \n \n ...,1539.0,384.0,35.0,2.27,24.95,15,1,De betalingstermijn van deze schuld is al vers...
3,IV,IV/NII/tekstblokken,NII055_00_NL.xml,NII055_00_NL,\n \n \n \n \n ...,1539.0,384.0,35.0,2.27,24.95,15,3,U hebt niet gereageerd op de aanmaning die ik ...
4,IV,IV/NII/tekstblokken,NII055_00_NL.xml,NII055_00_NL,\n \n \n \n \n ...,1539.0,384.0,35.0,2.27,24.95,15,4,Ook na betekening van een dwangbevel heb ik ge...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
37398,tekstblokken,tekstblokken,LH Betaalverzuimboeten.xml,LH Betaalverzuimboeten,Betaalverzuimboete(n)\n€\ntotaalbedrag_betaalv...,57.0,0.0,4.0,7.02,0.00,1,0,Betaalverzuimboete(n) € totaalbedrag_betaalver...
37399,DW,DW/RRZ/tekstblokken,RRZ119_00.xml,RRZ119_00,te_koop_aangeboden\n\n\nschuldenaar\n\n\nplaat...,95.0,0.0,14.0,14.74,0.00,1,0,te_koop_aangeboden schuldenaar plaats_van_...
37400,tekstblokken,tekstblokken,NII 10 Aanslaggegevens 01 en 02.xml,NII 10 Aanslaggegevens 01 en 02,Middel\n\nAanslagnummer\n\nDagtekening\n\n\n\n...,101.0,0.0,16.0,15.84,0.00,1,0,Middel Aanslagnummer Dagtekeningbelastingmid...
37401,IV,IV/NGR/tekstblokken,NGR120_01_Adres.xml,NGR120_01_Adres,Bewindvoerder\nnaam_bewindvoerder\n\nBurgerser...,103.0,0.0,8.0,7.77,0.00,1,0,Bewindvoerder naam_bewindvoerder Burgerservic...


In [19]:
no_dup_df[no_dup_df["FileCode"] =='DIV'].shape

(8, 13)

In [20]:
no_dup_df["SA Label"] = " " #to fill in label during the annotation experiment


/home/ADS/rinkl00_w00215/stageLR/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [21]:
no_dup_df.iloc[2929]['cleaned_text']

'Verlengen beslistermijn Ik streef ernaar om binnen acht weken na ontvangst van uw verzoek te beslissen.'

In [22]:
no_dup_df = no_dup_df[no_dup_df['cleaned_text'].str.len() > 50]
print(no_dup_df.shape)
print(no_dup_df.columns)
print()
print(set(no_dup_df["FileCode"].values))


(10254, 14)
Index(['FileCode', 'FilePath', 'FileName', 'FileCaption', 'Text', 'len_text',
       'whitespace_count', 'newline_count', 'perc_newline', 'perc_whitespace',
       'total_nr_sent', 'sent_index_in_tb', 'cleaned_text', 'SA Label'],
      dtype='object')

{'IB', 'ondertekeningen', 'BPM', 'AC', 'LH', 'DIV', 'IV', 'VPB', 'MM', 'tekstblokken', 'OB', 'DW', 'IV/INL', 'INL'}


### Extract sentences from three domains

In [23]:
df_OB = no_dup_df[no_dup_df["FileCode"] == 'OB']
print(df_OB.shape)
df_IV = no_dup_df[no_dup_df["FileCode"] == 'IV']
print(df_IV.shape)
df_INL = no_dup_df[no_dup_df["FileCode"] == 'INL']
print(df_INL.shape)

df_annotation = pd.concat([df_OB, df_IV])
df_annotation = df_annotation.sample(frac=1)
df_annotation.shape

(1414, 14)
(3790, 14)
(974, 14)


(5204, 14)

In [24]:
df_annotation["Sample_nr"] = ''
df_annotation= df_annotation[['FileCode','FileName', 'FileCaption', 'Sample_nr','Text', 'len_text',
       'whitespace_count', 'newline_count', 'perc_newline', 'perc_whitespace',
       'total_nr_sent', 'sent_index_in_tb', 'cleaned_text', 'SA Label']]


In [25]:
#df_annotation.to_csv("Full_annotatiesample.csv", sep=';')